## IBM Quantum Spring Challenge 2023 Lab 5

- Lab 5번 문제는 실제 하드웨어에서 127 qubit을 GHZ state로 만드는 문제
- 문제의 이해를 돕기 위해 간단한 27 qubit에 대한 예제를 설명
- 본 코드에서는 ibm_auckland 를 활용하였고 같은 구조의 다른 하드웨어(ibm_peekskil 등) 에서도 적용 가능


필요 정보 import

In [21]:
from typing import List, Optional
from qiskit import transpile, QuantumCircuit, QuantumRegister, ClassicalRegister
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")

import math


pi = math.pi

quantum_register = QuantumRegister(27)
classical_register = ClassicalRegister(27)

In [22]:
# Qubit의 기능을 설정

def qubit_index():
    initial_layout = [ 0, 1, 2, 3, 4, 5, 6, 7,  8, 9, 10, 11, 12, 13, 14, 15,16, 17, 18, 19, 20, 21, 22, 23, 24, 25,26 ]
    ghz_qubits = [1, 3, 7, 8, 12, 14, 18, 19, 23, 25    ]
    stabilizer_qubits = [0, 2, 4, 5, 6, 9, 10, 11, 13, 15, 16, 17, 20, 21, 22, 24, 26 ]
    return initial_layout, ghz_qubits, stabilizer_qubits

initial_layout, ghz_qubits, stabilizer_qubits = qubit_index()

# Exercise 1

문제 : 27 qubit GHZ state 를 ibm_auckland 하드웨어를 사용해서 생성하시오
- 실제 하드웨어에 적요될 경우 depth를 최소화하는것이 중요
- circuit 중앙에 Hadamard gate를 가하고 Hadamard gate를 가한 qubit 을 중심으로 cx gate를 적용


In [23]:
def generate_ghz27(quantum_register, classical_register):
    
    # Quantum circuit 생성
    qc = QuantumCircuit(quantum_register, classical_register)

    # 13번 Qubit에 Hadamard gate를 만듦
    qc.h(13)

    cx_g = [[13,12],
            [13,14],
            [12,10],
            [10,7],
            [7,4],
            [4,1],
            [1,0],
            [1,2],
            [7,6],
            [12,15],
            [15,18],
            [18,21],
            [21,23],
            [23,24],
            [18,17],
            [14,16],
            [16,19],
            [19,22],
            [22,25],
            [25,26],
            [19,20],
            [14,11],
            [11,8],
            [8,5],
            [5,3],
            [8,9]]
            
    # 나머지 qubit에 CX gate 적용
    for i in range(len(cx_g)):
        qc.cx(cx_g[i][0],cx_g[i][1])

    return qc


In [24]:
# 결과 확인
ghz_circuit = generate_ghz27(quantum_register, classical_register)
print("27 qubit circuit depth : ",ghz_circuit.depth())

27 qubit circuit depth :  8


# Exercise 2

문제 :  stabilizer_qubitt에 해당하는 qubit의 얽힘을 풀고 GHZ state를 만드시오
- 하드웨어상에서 실제로 연결되어 있는 qubit에 대해 cx gate를 가하는것이 중요

In [25]:
def deentangle_qubits(quantum_register, classical_register):
    qc = QuantumCircuit(quantum_register, classical_register)

    # 연결된 qubit 간에 cx gate를 가함
    qc.cx(1,0)
    qc.cx(1,2)
    qc.cx(7,4)
    qc.cx(7,6)
    qc.cx(12,10)
    qc.cx(18,15)
    qc.cx(18,17)
    qc.cx(23,21)
    qc.cx(23,24)
    qc.cx(25,26)
    qc.cx(25,22)
    qc.cx(19,16)
    qc.cx(19,20)
    qc.cx(14,13)
    qc.cx(8,9)
    qc.cx(8,11)
    qc.cx(3,5)

    return qc

In [26]:
# 결과 확인
unentangle_circuit = deentangle_qubits(quantum_register, classical_register)

print("Circuit depth : ", unentangle_circuit.depth())

Circuit depth :  2


Exercise 1과 2을 수행후 qubit을 측정하는 Quantim circut을 생성

In [27]:
# Stabilizer qubit을 측정
def measure_stabilizers(quantum_register, classical_register, stabilizer_qubits):
    qc = QuantumCircuit(quantum_register, classical_register)
    qc.measure(stabilizer_qubits, stabilizer_qubits)

    return qc
stabilizer_circuit = measure_stabilizers(quantum_register, classical_register, stabilizer_qubits)

# GHZ qubit을 측정
def measure_ghz(quantum_register, classical_register, ghz_qubits):
    qc = QuantumCircuit(quantum_register, classical_register)
    qc.measure(ghz_qubits, ghz_qubits)

    return qc

measure_circuit = measure_ghz(quantum_register, classical_register, ghz_qubits)

# Exercise1, 2, 측정을 수행하는 circuit을 합침
simple_ghz = (
    ghz_circuit.compose(unentangle_circuit)
    .compose(stabilizer_circuit)
    .compose(measure_circuit)
)

실제 하드웨어에 적용

In [28]:
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()

# Backend 설정
backend = provider.get_backend("ibm_auckland")

# 하드웨어에 적합한 quantum circuit으로 tranpile
qc_transpiled = transpile(simple_ghz, backend, initial_layout=initial_layout)


# 하드웨어에서 qc_transpiled 를 연산함
number_of_shots: int = 1024
job = backend.run(
    qc_transpiled,
    shots=number_of_shots,
    memory=True
)

# 결과를 가져옴
data = job.result().get_memory()

# Ecercise 3

문제 : GHZ state를 평가할 수 있는 함수를 만드시오
- 이상적인 결과는 모두 0 또는 모두 1인 경우
- 이상적인 GHZ state일수록 값이 작아야함

In [29]:
def test_ghz(data, ghz_qubits):
    
    quality = []
    
    # 모든 shot에 대해 수행
    for i in range(len(data)):
        zero_num = 0
        qubit_num = 0
        
        # shot에 있는 모든 qubit에 대하여
        for j in data[i]:
            
            # Ghz_qubit에 해당하는 경우에만 고려
            if qubit_num in ghz_qubits:
                if j =="0":
                    zero_num = zero_num+1
        
            qubit_num = qubit_num+1
        
        # 0이 많이 측정된 경우
        if zero_num > len(ghz_qubits)/2:
            
            # 좋은 GHZ state일수록 값이 작아야하므로 보정
            zero_num = len(ghz_qubits)-zero_num
        quality.append(zero_num)

    # 평균, 최대, 최소에 대해 print
    mean = sum(quality)/ len(quality)
    print("max:", max(quality))
    print("min:", min(quality))
    print("mean:", mean)
    
    # 평균값을 return
    return mean
    


In [30]:
# 결과 확인

test_ghz(data, ghz_qubits)

max: 5
min: 0
mean: 1.8271484375


1.8271484375